# 01 - Is it a Bird?

For an introduction to AI, we will follow an example from [fast.ai](https://github.com/fastai/course22/blob/master/00-is-it-a-bird-creating-a-model-from-your-own-data.ipynb), even though we don't have all the internal knowlewdge, seeing it in action can be eye opening.

In 2015 the idea of creating a computer system that could recognise birds was considered so outrageously challenging that it was the basis of [this XKCD joke](https://xkcd.com/1425/).

![image.png](https://imgs.xkcd.com/comics/tasks.png)

We'll do exactly that, in just a few minutes, by:

- Use DuckDuckGo to search for images of "bird photos"
- Use DuckDuckGo to search for images of "forest photos"
- Fine-tune a pretrained neural network to recognise these two groups
- Try running this model on a picture of a bird and see if it works.

## About these notebooks

This is a Jupyter notebook. Notice that we can have both prose, like this explanation, and code, like the cell below. To execute the cell below, select it and press the ▶️ (*Run this cell and Advance*) icon in the toolbar.

In [ ]:
print('Hello World!')

The shortcut for execution is `Shift` + `Enter`. Try that in the cell below.

In [ ]:
print('5 + 7 =', 5+7) 

Optionally, you can press the ⏩ (*Restart the kernel and run all cells*) icon to execute the whole notebook at once. Since this is a learning and not a production notebook, we recommend you step one cell at a time instead.

Notice that you can execute cells in any order, or multiple times. The history of cell execution is labeled to the left of each cell, like \[1\] indicating this is the first cell that was executed. Most notebooks expect a linear progression, but you can re-run any cell at any time.

## Step 1: Download images of birds and non-birds

We use the duck duck go search engine to look for photos, get their URL, and then doownload the images.

In [ ]:
from duckduckgo_search import DDGS
from fastcore.all import *

# We need to redefine this fast.ai function to work with newer versions of duckduckgo_search
def search_images(term, max_images=200):
    with DDGS(headers = {'Accept-Encoding': 'gzip, deflate, br'}) as ddgs:
        results = ddgs.images(keywords=term)
        images = [result.get("image") for result in results[:max_images]] #  https://stackoverflow.com/a/78327127
        #images = [next(results).get('image') for _ in range(max_images)] # Gives iterator error
    return L(images)

In [ ]:
urls = search_images('bird photos', max_images=1) # Get a list of images, size 1
urls[0] # Print URL

In [ ]:
# Save the file
from fastdownload import download_url
dest = 'data/bird.jpg'
download_url(urls[0], dest, show_progress=False)

In [ ]:
# Show the saved image file
from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Now let's do the same with "forest photos"

In [ ]:
download_url(search_images('forest photos', max_images=1)[0], 'data/forest.jpg', show_progress=False)
Image.open('data/forest.jpg').to_thumb(256,256)

Let's grab 200 examples of each of "bird" and "forest" photos, and save each group of photos to a different folder. This might take a while. If `[*]` is next to the cell, it means the cell is still executing.

In [ ]:
## First, let's delete any existing images, otherwise we might run into out of memory errors when trying to fit them all at once
import os
import glob

files = glob.glob('data/bird_or_not/bird/*')
for f in files:
    os.remove(f)
files = glob.glob('data/bird_or_not/forest/*')
for f in files:
    os.remove(f)

In [ ]:
searches = 'forest','bird'
path = Path('data/bird_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

## Step 2: Train our model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them.

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). With `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

Here is what each of the `DataBlock` parameters means:

    blocks=(ImageBlock, CategoryBlock),

The inputs to our model are images, and the outputs are categories (in this case, "bird" or "forest").

    get_items=get_image_files,
    
To find all the inputs to our model, run the `get_image_files` function (which returns a list of all image files in a path).

    splitter=RandomSplitter(valid_pct=0.2, seed=42),

Split the data into training and validation sets randomly, using 20% of the data for the validation set.

    get_y=parent_label,

The labels (`y` values) is the name of the `parent` of each file (i.e. the name of the folder they're in, which will be *bird* or *forest*).

    item_tfms=[Resize(192, method='squish')]
    
Before training, resize each image to 192x192 pixels by \"squishing\" it (as opposed to cropping it).


Now we're ready to train our model. The fastest widely used computer vision model is `resnet18`. You can train this in a few minutes, even on a CPU! (On a GPU, it generally takes under 10 seconds...)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

"Fine-tuning" a model means that we're starting with a model someone else has trained using some other dataset (called the *pretrained model*), and adjusting the weights a little bit so that the model learns to recognise your particular dataset. In this case, the pretrained model was trained to recognise photos in imagenet, and widely-used computer vision dataset with images covering 1000 categories) 

## Step 3:

Let's see what our model thinks about that bird we downloaded at the start:

In [ ]:
is_bird,_,probs = learn.predict(PILImage.create('data/bird.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")

You've created a binary classifier. Good job!

**Next Notebook: [02-Datasets](02-Datasets.ipynb)**